In [459]:
import pandas as pd
import json
import numpy as np
from scipy import stats
import warnings
import math

In [460]:
path = "/Users/geovern/Downloads"

In [461]:
mqm_scores = pd.read_csv(filepath_or_buffer="{}/mqm_ted_zhen.tsv".format(path), header=0, sep="\t", error_bad_lines=False)

/var/folders/6z/mxnyjlrx4tv5y74x80k3_cwcvylpqt/T/ipykernel_28664/3573112560.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  mqm_scores = pd.read_csv(filepath_or_buffer="{}/mqm_ted_zhen.tsv".format(path), header=0, sep="\t", error_bad_lines=False)


In [462]:
mqm_scores

,system,doc,doc_id,seg_id,rater,source,target,category,severity
0,Borderline,talk.2,1,84,rater3,我希望大家能花点时间考虑一个非常简单的事实那就是，到目前为止， 我们对宇宙的大部分了解都来自于光。,I want you to <v>take a moment</v> to consider...,Style/Awkward,Major
1,Borderline,talk.2,1,84,rater3,我希望大家能花点时间考虑一个非常简单的事实那就是，到目前为止， 我们对宇宙的大部分了解都来自于光。,I want you to take a moment to consider <v>the...,Fluency/Grammar,Major
2,Borderline,talk.2,1,84,rater3,我希望大家能花点时间考虑一个非常简单的事实那就是，到目前为止， 我们对宇宙的大部分了解都来自于光。,I want you to take a moment to consider the ve...,Style/Awkward,Major
3,Borderline,talk.2,1,84,rater3,我希望大家能花点时间考虑一个非常简单的事实那就是，到目前为止， 我们对宇宙的大部分了解都来自于光。,I want you to take a moment to consider the ve...,Accuracy/Mistranslation,Major
4,DIDI-NLP,talk.2,1,84,rater7,我希望大家能花点时间考虑一个非常简单的事实那就是，到目前为止， 我们对宇宙的大部分了解都来自于光。,I hope you can take some time to consider the ...,No-error,No-error
...,...,...,...,...,...,...,...,...,...
9910,metricsystem3,talk.9,159,843,rater5,（掌声）,(Applause),No-error,No-error
9911,metricsystem4,talk.9,159,843,rater7,（掌声）,(applause),No-error,No-error
9912,metricsystem5,talk.9,159,843,rater2,（掌声）,(<v>Applause</v>),Fluency/Grammar,Minor
9913,ref,talk.9,159,843,rater2,（掌声）,(<v>Applause</v>),Fluency/Grammar,Minor


In [463]:
context_errors = {}
num_errors = 0
for index, row in mqm_scores.iterrows():
    if "context" in row["category"] and "ref" not in row["system"]:
        system = row["system"].split(".")[-1] if "hyp" in row["system"] else row["system"].replace(".", "")
        ind = row["seg_id"]
        num_errors += 1
        if system in context_errors:
            if ind-1 not in context_errors[system]:
                context_errors[system].append(ind - 1)
        else:
            context_errors[system] = [ind -1]
print("There are {} contex-related errors in the dataset".format(num_errors))

There are 163 contex-related errors in the dataset


In [464]:
context_errors = {k: sorted(v) for k,v in context_errors.items()}

In [465]:
filepath = '/Users/geovern/Documents/'

In [466]:
campaign = "wmt21.tedtalks"
lp = "zh-en"

In [468]:
with open(filepath + "scores/" + '{}_{}_seg-comet21-qe_scores.json'.format(campaign, lp), "r") as f:
    scores = json.load(f)    
    context_errors_scores = {}
    for system in context_errors:
        context_errors_scores[system] = np.array(scores["sys"][system][0])[context_errors[system]]
with open(filepath + "scores/" '{}_{}_seg-ctxpartcomet21-qe_scores.json'.format(campaign, lp), "r") as f:
    scores_ctx = json.load(f)
    context_errors_scores_ctx = {}
    for system in context_errors:
        context_errors_scores_ctx[system] = np.array(scores_ctx["sys"][system][0])[context_errors[system]]

In [469]:
with open(filepath + "mqm_scores/" + '/{}_{}_mqm_scores.json'.format(campaign, lp), "r") as f:
    scores_mqm = json.load(f)
    context_errors_mqm = {}
    for system in context_errors:
        context_errors_mqm[system] = np.array(scores_mqm[system])[context_errors[system]]

In [470]:
with open(filepath + "mqm_scores/" + '/{}_{}_mqm_scores.json'.format(campaign, lp), "r") as f:
    scores_mqm = json.load(f)

In [471]:
list_context_errors_mqm, list_context_errors_scores, list_context_errors_scores_ctx = [], [], []
for system in context_errors_mqm:
    list_context_errors_mqm += list(context_errors_mqm[system])
    list_context_errors_scores += list(context_errors_scores[system])
    list_context_errors_scores_ctx += list(context_errors_scores_ctx[system])

In [472]:
class CorrFunction:
  """Wrap any correlation from scipy.stats, with optional averaging and Nones.

  This is a functor with the same interface as the scipy.stats functions:
  call with two float vectors, returns a correlation and pvalue.

  Averaging requires that you construct with num_sys > 0, indicating that input
  vectors will contain num_sys sets of item scores grouped by system. The
  returned correlation and pvalue are averages over per-item correlations. Short
  per-item lists can lead to repeated scores that make correlations undefined;
  these are discarded, and the Corr() function returns the number of items that
  were actually used (last value of the returned triple).

  Optional None-filtering checks for None entries in the first vector argument
  only (assumed to represent gold scores), and filters both vectors in tandem
  before computing correlations. When used with averaging, this can result in
  empty vectors and undefined correlations that are handled as described above.
  """

  def __init__(self, corr_fcn, num_sys=0, filter_nones=False):
    self._corr_fcn = corr_fcn
    self._num_sys = num_sys
    self._filter_nones = filter_nones

  def __call__(self, vect1, vect2):
    return self.Corr(vect1, vect2)[:2]

  def Corr(self, vect1, vect2):
    """Return correlation, pvalue, and number of items used for averaging."""
    # Reshape into item x system score matrices, for average over items.
    num_sys = self._num_sys or len(vect1)
    print(num_sys)
    mat1 = np.asarray(vect1).reshape(num_sys, -1).transpose()
    mat2 = np.asarray(vect2).reshape(num_sys, -1).transpose()
    tot_corr, tot_pval, n = 0, 0, 0
    with warnings.catch_warnings():
      warnings.simplefilter('ignore')
      for r1, r2 in zip(mat1, mat2):
        if self._filter_nones:
          filt = [(v1, v2) for v1, v2 in zip(r1, r2) if v1 is not None]
          if not filt: continue
          r1, r2 = zip(*filt)
        cv, pv = self._corr_fcn(r1, r2)
        if not math.isnan(cv):
          tot_corr += cv
          tot_pval += pv
          n += 1
    return (tot_corr / n, tot_pval / n, n) if n else (0, 0, 0)


In [473]:
none_count = list_context_errors_mqm.count(None)
cf = CorrFunction(stats.kendalltau, 0, none_count)
cf.Corr(list_context_errors_mqm, list_context_errors_scores)[0]

141


0.18305638195828625

In [474]:
cf.Corr(list_context_errors_mqm, list_context_errors_scores_ctx)[0]

141


0.1323659402259121